In [1]:
import pandas as pd
import numpy as np
import sys
import pickle
import warnings
warnings.filterwarnings("ignore")

sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset

import FineTuneModule
import EvaluationModule

import xgboost as xgb
import sklearn
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.pipeline import Pipeline

(CVXPY) Jul 12 05:18:10 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 12 05:18:10 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 12 05:18:10 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="binary_classification", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='SC', #option: KVB, SC
    imputation='IterativeImputer', #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    remove_low_DAS = True,
    save_csv=False,
    balance_class=True,
    random_state=2022)

feature engineering, drop columns due to 70% missing value: Index(['smkyrs', 'numcigs', 'rfstatus_impute', 'ccpstatus_impute',
       'statin_use'],
      dtype='object')
Missing values in train before imputation: 128
Missing values in train after imputation: 0
Missing values in test before imputation: 24
Missing values in test after imputation: 0
Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)
Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)
before balance class: 1.0    231
0.0     73
Name: DrugResponse_binary, dtype: int64
after balance class: 1.0    204
0.0    184
Name: DrugResponse_binary, dtype: int64
save file to: /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/.csv_temp/Train.csv
save file to: /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/.csv_temp/Test.csv


In [6]:
# read train, test from dataloader
train_set, train_loc = dataset.get_train()
test_set, test_loc = dataset.get_test()

In [8]:
train = train_set
test = test_set

In [9]:
aml = EvaluationModule.AutoBuild(seed=dataset.random_state, project_name="SC_Jul12", challenge=dataset.challenge)

In [20]:
model = fine_tune_rf(train=train, model="drf_classification", search_methods="RandomSearch")

param_search_grid: {'max_samples': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, None], 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
best_params: {'n_estimators': 1300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_samples': None, 'max_features': 'log2', 'max_depth': 110, 'bootstrap': True}


In [21]:
aml.validate("rf", model, train, test)

In [22]:
aml.validation

,model,MAE,MSE,RMSE,R2,Pearson_Correlation,Accuracy,F1-Score
0,rf,None,None,None,None,None,0.871795,0.871795
1,rf,None,None,None,None,None,0.820513,0.819444
2,rf,None,None,None,None,None,0.897436,0.886710
3,rf,None,None,None,None,None,0.833333,0.825022
4,rf,None,None,None,None,None,0.846154,0.845748
5,rf,None,None,None,None,None,0.858974,0.858765
6,rf,None,None,None,None,None,0.884615,0.882274
7,rf,None,None,None,None,None,0.884615,0.883058
8,rf,None,None,None,None,None,0.730769,0.730725
9,rf,None,None,None,None,None,0.858974,0.857829


In [23]:
np.mean(aml.validation[aml.validation['model'] == 'rf']['Accuracy'])

0.8487179487179487

In [24]:
aml.test

,model,MAE,MSE,RMSE,R2,Pearson_Correlation,Accuracy,F1-Score
10,rf,None,None,None,None,None,0.694118,0.499547
